<a href="https://colab.research.google.com/github/nuot/AL-Clean/blob/main/AssistantListing_ObliClean_InProgress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assistant Listing Obligation data cleaning project In Progress** 
<br>

Author: Nuo Tian



**Last Updated: 03/14/2021**

In [1139]:
import pandas as pd
import re
data = pd.read_csv('AssistanceListings.csv')

In [1140]:
obligations = data['Obligations (122)']

In [1141]:
types = []
for row in obligations:
  a = re.findall(r'\((.*?)\) FY',str(row))
  if a:
    types.append(a)
  else:
    types.append('None')


In [1142]:
types_dedup = []
for type1 in types:
    type1 = list(dict.fromkeys(type1))
    types_dedup.append(type1)

In [1143]:
data['Obligation Types'] = types_dedup

**Checking Point** 
<br>
Done with obligation sepration
<br>
Next working on seprating the $amount for each obligation type

In [1144]:
def listToStringWithoutBrackets(list1):
    return str(list1).replace('[','').replace(']','').replace("'","")

In [1145]:
Obli_types = []
for i in range(len(data['Obligation Types'])):
  res = listToStringWithoutBrackets(data['Obligation Types'][i])
  Obli_types.append(res)

In [1146]:
data['Obligation Types'] = Obli_types

In [1147]:
maxlen = max(list(map(lambda x: len(x.split(",")) ,data['Obligation Types'])))
cols = ["Obligation"+str(x)   for x in range(maxlen)]
datalist = list(map(lambda x: x.split(","), data['Obligation Types']))
newdf = pd.DataFrame(data=datalist, columns=cols) 
data = pd.concat([data, newdf], axis=1) 

In [1148]:
mask = data.applymap(lambda x: x is None)
cols = data.columns[(mask).any()]
for col in data[cols]:
    data.loc[mask[col], col] = ''

Amount

In [1149]:
df = pd.DataFrame()

i = 0
for i in range(len(obli1.columns)):
  val = []
  for row in obli1[i]:
    a = re.findall(r'FY [0-9][0-9]\$[^\]]+[.%]?',str(row))
    b = re.findall(r'FY [0-9][0-9] est \$[^\]]+[.%]?',str(row))
    if a:
      val.append(a)
    elif b:
      val.append(b)
    else:
      val.append('')
  df.insert(i, i, val, True) 


In [1150]:
df = df.applymap(str)

In [1151]:
import string

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace("['", "")
        text = text.replace("- ']", "")
    return text

In [1152]:
i = 0
while i < len(df.columns):
  df[i] = df[i].apply(remove_punctuations)
  i +=1

In [1153]:
for j in range(maxlen):
  FY19 = []
  FY20 = []
  FY20_Flag = []
  FY21 = []
  FY21_Flag = []
  name = 'Obligation'+str(j)
  for i in range(len(df[j])):  
    a = re.findall(r'FY 19\$[^\]]+[.%]?',df[j+1][i])
    b = re.findall(r'FY 20\$[^\]]+[.%]?',df[j+1][i])
    c = re.findall(r'FY 20 est \$[^\]]+[.%]?', df[j+1][i])
    d = re.findall(r'FY 21\$[^\]]+[.%]?',df[j+1][i])
    e = re.findall(r'FY 21 est \$[^\]]+[.%]?', df[j+1][i])
    if a :
      FY19.append(a[0].split(';')[0].replace('FY 19$',''))
    else:
      FY19.append('')
    if b:
      FY20.append(b[0].split(';')[0].replace('FY 20$',''))
      FY20_Flag.append(0)
    elif c:
      FY20.append(c[0].split(';')[0].replace('FY 20 est $',''))
      FY20_Flag.append(1)
    else:
      FY20.append('')
      FY20_Flag.append('')
    
    if d:
      FY21.append(d[0].split(';')[0].replace('FY 21$',''))
      FY21_Flag.append(0)
    elif e:
      FY21.append(e[0].split(';')[0].replace('FY 21 est $',''))
      FY21_Flag.append(1)
    else:
      FY21.append('')
      FY21_Flag.append('')

  data.insert(data.columns.get_loc(name)+1, 'FY21_Flag'+ str(j), FY21_Flag)
  data.insert(data.columns.get_loc(name)+1, 'FY21_'+ str(j), FY21)
  data.insert(data.columns.get_loc(name)+1, 'FY20_Flag'+ str(j), FY20_Flag)
  data.insert(data.columns.get_loc(name)+1, 'FY20_'+ str(j), FY20)
  data.insert(data.columns.get_loc(name)+1, 'FY19_'+ str(j), FY19)


In [1154]:
data.head(20)

,Program Title,Program Number,Popular Name (020),Federal Agency (030),Authorization (040),Objectives (050),Types of Assistance (060),Uses and Use Restrictions (070),Applicant Eligibility (081),Beneficiary Eligibility (082),Credentials/Documentation (083),Preapplication Coordination (091),Application Procedures (092),Award Procedure (093),Deadlines (094),Range of Approval/Disapproval Time (095),Appeals (096),Renewals (097),Formula and Matching Requirements (101),Length and Time Phasing of Assistance (102),Reports (111),Audits (112),Records (113),Account Identification (121),Obligations (122),Range and Average of Financial Assistance (123),Program Accomplishments (130),"Regulations, Guidelines, and Literature (140)",Regional or Local Office (151),Headquarters Office (152),Website Address (153),Related Programs (160),Examples of Funded Projects (170),Criteria for Selecting Proposals (180),Published Date,Parent Shortname,URL,Recovery,Obligation Types,Obligation0,FY19_0,FY20_0,FY20_Flag0,FY21_0,FY21_Flag0,Obligation1,FY19_1,FY20_1,FY20_Flag1,FY21_1,FY21_Flag1,Obligation2,FY19_2,FY20_2,FY20_Flag2,FY21_2,FY21_Flag2,Obligation3,FY19_3,FY20_3,FY20_Flag3,FY21_3,FY21_Flag3,Obligation4,FY19_4,FY20_4,FY20_Flag4,FY21_4,FY21_Flag4,Obligation5,FY19_5,FY20_5,FY20_Flag5,FY21_5,FY21_Flag5,Obligation6,FY19_6,FY20_6,FY20_Flag6,FY21_6,FY21_Flag6,Obligation7,FY19_7,FY20_7,FY20_Flag7,FY21_7,FY21_Flag7,Obligation8,FY19_8,FY20_8,FY20_Flag8,FY21_8,FY21_Flag8,Obligation9,FY19_9,FY20_9,FY20_Flag9,FY21_9,FY21_Flag9,Obligation10,FY19_10,FY20_10,FY20_Flag10,FY21_10,FY21_Flag10
0,Agricultural Research Basic and Applied Research,10.001,(Extramural Research),"AGRICULTURAL RESEARCH SERVICE, AGRICULTURE, DE...","{""list"":[{""act"":{""description"":""Food Security ...","To make agricultural research discoveries, eva...",PROJECT GRANTS,Not Applicable,Usually nonprofit institutions of higher educa...,Usually nonprofit institutions of higher educa...,"{""isApplicable"":false}","{""environmentalImpact"":{""reports"":[{""isSelecte...","{""description"":""Letters should be submitted to...","A peer review panel considers each proposal, e...","{""flag"":""no"",""list"":[]}",Not Applicable,None,None,"{""types"":{""moe"":false,""formula"":false,""matchin...","{""awarded"":""other"",""description"":""None"",""award...","[{""code"":""program"",""isSelected"":true,""descript...","{""isApplicable"":true,""description"":""As perform...","Financial records, supporting documents, stati...",12-1400-0-1-352;,"(Project Grants) FY 19$4,704,045.00; FY 20 est...","$5,000 to $50,000. Average $20,000","{""list"":[],""isApplicable"":false}",2 CFR 200,"{""flag"":""appendix"",""description"":""See the Agri...","Deborah Watson5601 Sunnyside Avenue, MS 5110, ...",http://www.ars.usda.gov,10.207 Animal Health and Disease Research; 10....,Not Applicable.,Peer review.,"Jan 01,1965",USDA,https://beta.sam.gov/fal/2cf5bced6e734d309ddad...,No,Project Grants,Project Grants,"4,704,045.00","6,617,316.00",1,"7,000,000.00",1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,"Plant and Animal Disease, Pest Control, and An...",10.025,NaN,"ANIMAL AND PLANT HEALTH INSPECTION SERVICE, AG...","{""list"":[{""act"":{""description"":""Plant Protecti...",To protect U.S. agriculture from economically ...,PROJECT GRANTS,Not Applicable,"Foreign, State, local, and U.S. Territorial go...","Farmers, ranchers, agriculture producers, Stat...","{""description"":""Curriculum vitae for principal...","{""environmentalImpact"":{""reports"":[{""isSelecte...","{""description"":""Comply with E.O. 12372, \""Inte...",Applications are approved by the Administrator...,"{""flag"":""contact"",""list"":[]}",From 60 to 120 days.,None,Based on program needs and availability of ann...,"{""types"":{""moe"":false,""formula"":false,""matchin...","{""awarded"":""other"",""description"":""Up to 1 year...","[{""code"":""program"",""isSelected"":false,""descrip...","{""isApplicable"":true,""description"":""N/A""}",Instruction provided 

In [1155]:
data
data.to_csv('AssistanceListingFinal.csv')

from google.colab import files
files.download('AssistanceListingFinal.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>